In [ ]:
import tensorflow as tf
from tensorflow import keras

from logic.dataset import Dataset
from logic.model import create_model

## load data

In [ ]:
train_filepath = "../data/train.csv"
valid_filepath = "../data/valid.csv"
test_filepath = "../data/test.csv"

train_set = Dataset.csv_to_np_dataset(train_filepath)
valid_set = Dataset.csv_to_np_dataset(valid_filepath)
test_set = Dataset.csv_to_np_dataset(test_filepath)

## model (single input, single output)

In [ ]:
model = create_model("singleinput_singleoutput")
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train = train_set[:,:-1]
y_train = train_set[:,-1]
X_valid = valid_set[:,:-1]
y_valid = valid_set[:,-1]
X_test = test_set[:,:-1]
y_test = test_set[:,-1]

batch_size = 32
len_train = sum([1 for _ in open('../data/train.csv')])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
loss = model.evaluate(X_test, y_test)

import matplotlib.pyplot as plt
plt.plot(history.history["loss"])

## model (multiinput, multi output)

In [ ]:
model = create_model("multiinput_multioutput")
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = train_set[:,:5], train_set[:,2:-1]
y_train = train_set[:,-1]
X_valid_A, X_valid_B = valid_set[:,:5], valid_set[:,2:-1]
y_valid = valid_set[:,-1]
X_test_A, X_test_B = test_set[:,:5], test_set[:, 2:-1]
y_test = test_set[:,-1]

batch_size = 32
len_train = sum([1 for _ in open('../data/train.csv')])

history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20, validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))
loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

import matplotlib.pyplot as plt
plt.plot(history.history["loss"])